In [2]:
!mkdir data
import os
for i in range(10):
  os.system(f'wget https://cernbox.cern.ch/remote.php/dav/public-files/YQxujEYrVFFpylN/batch_1_{i}.tar.gz')
  os.system(f'tar -xzf /content/batch_1_{i}.tar.gz')
  os.system(f'cp ./batch_1_{i}/*.pt ./data')
  os.system(f'rm -rf batch_1_{i}')

In [3]:
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"

1.13.1+cu116
11.6


In [4]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.13.1+$11.6.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.13.1+$11.6.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.13.1+$11.6.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.13.1+$11.6.html
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.1+1.6.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 KB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.0-cp39-cp39-linux_x86_64.whl size=485242 sha256=03643edd0e560c142361491b2fd6b1aa898af32a5330eb79107d8ed7e37b1a81
  Stored in directory: /root/.cache/pip/wheels/c5/33/3c/b02defb8e41252b9073b3b98433e082a8fb9aa8945127ffcbe
Successfully built torch-scatter
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.1+1.6.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.2/208.2 KB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-sparse: filename=torch_sparse-0.6.16-cp39-cp39-linux_x86_64.whl size=

In [6]:
import torch
BATCH_SIZE=32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
from pathlib import Path
from torch.utils.data import Dataset  # not the one from PyG!
from torch_geometric.loader import DataLoader
import torch
import numpy as np
from sklearn.model_selection import train_test_split

class MyDataset(Dataset):
  def __init__(self, path: Path, lists:bool=False, data:list=[]):
    super().__init__()
    if lists:
      self.graphs = data
    else:
      self.graphs = list(path.glob("*.pt"))

  def __getitem__(self, idx):
    return torch.load(self.graphs[idx]).to(device)
  
  def __len__(self) -> int:
    return len(self.graphs)

data=list(Path('./data').glob('*.pt'))
train,test = train_test_split(data,train_size=0.8)

dataset = MyDataset(Path("./data"))
trainset = MyDataset('', True, train)
testset = MyDataset('', True, test)
loader = DataLoader(dataset, batch_size=BATCH_SIZE)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE)
testloader = DataLoader(testset, batch_size=BATCH_SIZE)

In [8]:
print(trainset[0])
print(trainset[0].x)
print(trainset[0].edge_index)
print(trainset[0].edge_attr)
print(trainset[0].y)

Data(x=[176, 6], edge_index=[2, 1270], edge_attr=[1270, 4], y=[1270])
tensor([[ 0.1454, -0.2843, -1.5020, -3.0306,  4.3126, -5.3583],
        [ 0.1583, -0.2856, -1.5020, -2.9458,  3.9399, -4.9366],
        [ 0.1462, -0.2862, -1.5020, -3.0254,  4.2574, -5.3560],
        ...,
        [ 0.1311, -0.2866,  1.5020,  3.1333,  4.7371, -5.9755],
        [ 0.1713, -0.2358,  1.4980,  2.8648,  4.3074, -3.9400],
        [ 0.1718, -0.2358,  1.5020,  2.8647,  4.2954, -3.9291]])
tensor([[  0,   0,   1,  ..., 175, 174, 175],
        [  4,   5,   6,  ..., 169, 170, 170]])
tensor([[-0.0198, -0.0008,  0.2000,  0.0042],
        [-0.0189, -0.0022,  0.2000,  0.0080],
        [-0.0202, -0.0011,  0.2000,  0.0071],
        ...,
        [-0.0245,  0.0017, -0.2040,  0.0094],
        [-0.0235,  0.0016, -0.1960,  0.0091],
        [-0.0240,  0.0016, -0.2000,  0.0092]])
tensor([1., 0., 1.,  ..., 1., 1., 1.])


In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils
from torch_geometric.utils import negative_sampling

In [10]:
torch.set_printoptions(linewidth=200)


In [23]:
class GNNStack(nn.Module):
  def __init__(self,input_dim:int,hidden_dim:int,output_dim:int,num_edge_feat,layers:int,dropout:bool=False):
    super(GNNStack,self).__init__()
    self.dropout = dropout
    self.layers = layers
    self.convs = nn.ModuleList()
    # nn.Linear(num_edge_feat,input_dim*hidden_dim)
    self.convs.append(pyg_nn.NNConv(input_dim,hidden_dim,nn.Sequential(nn.Linear(num_edge_feat, 32), nn.ReLU(),nn.Linear(32, input_dim * hidden_dim))))
    if layers>1:
      for i in range(layers-1):
        self.convs.append(pyg_nn.NNConv(hidden_dim,hidden_dim,nn.Sequential(nn.Linear(num_edge_feat, 32), nn.ReLU(),nn.Linear(32, hidden_dim * hidden_dim))))
    self.post_conv = nn.Sequential(nn.Linear(hidden_dim,hidden_dim),nn.Dropout(self.dropout*0.5),nn.Linear(hidden_dim,output_dim))

  def forward(self,x,edge_index,edge_attr):
    for i in range(len(self.convs)):
      x = self.convs[i](x,edge_index,edge_attr)
      x = F.relu(x)
      x = F.dropout(x, self.dropout*0.5,training= self.training)
      # print(f'stage{i}',x)
    x = self.post_conv(x)
    # print(x,x.size())
    return x

  def predict_link(self,x,edge_index):
    # print('pred link',edge_index[0],edge_index[1],edge_index[1].size())
    # print(x[edge_index[0]],x[edge_index[0]].size(),x[edge_index[1]].size())
    # print('link',(x[edge_index[0]]*x[edge_index[1]]).sum(dim=-1).size())
    ret = (x[edge_index[0]]*x[edge_index[1]]).sum(dim=-1)
    # ret = (10*ret)/torch.max(ret)
    return ret

  def loss(self, prediction, label):
    return F.nll_loss(prediction, label)

In [24]:
def train(model, trainset, optim, criterion):

  model.train()
  optim.zero_grad()
  train_losses = []
  for i in range(len(trainset)):
    neg_edge_index = negative_sampling(edge_index=trainset[i].edge_index, num_nodes=trainset[i].num_nodes,
                                        num_neg_samples=trainset[i].edge_index.size(1),method='sparse')
    # print('nei',neg_edge_index,neg_edge_index.size())
    edge_index = torch.cat([trainset[i].edge_index,neg_edge_index],dim=-1)
    y = torch.cat([trainset[i].y,trainset[i].y.new_zeros(neg_edge_index.size(1))],dim=0)

    z= model.forward(trainset[i].x,trainset[i].edge_index,trainset[i].edge_attr)
    # print('z',z)
    # print('y',y,y.size())
    # print(edge_index,edge_index.size(),trainset[i].edge_index.size())
    out = model.predict_link(z,edge_index).view(-1)
    # print(out,y)
    loss = criterion(out, y)
    train_losses.append(loss)
    loss.backward()
    optim.step()
  # print(sum(train_losses)/len(train_losses))
  return sum(train_losses)/len(train_losses)

In [25]:
from sklearn.metrics import roc_auc_score
import numpy as np

model = GNNStack(input_dim=trainset[0].num_features,hidden_dim=32,output_dim=16,num_edge_feat=trainset[0].num_edge_features,layers=1).to(device)
optim = torch.optim.Adam(params=model.parameters(),lr=0.01)
criterion = nn.BCEWithLogitsLoss() 

In [26]:
def test(model,testset):
  model.eval()
  test_scores=[]
  for i in range(len(testset)):
    neg_edge_index = negative_sampling(edge_index=testset[i].edge_index, num_nodes=testset[i].num_nodes,
                                        num_neg_samples=testset[i].edge_index.size(1),method='sparse')
    edge_index = torch.cat([testset[i].edge_index,neg_edge_index],dim=-1)
    y = torch.cat([testset[i].y,testset[i].y.new_zeros(neg_edge_index.size(1))],dim=0)
    z = model.forward(testset[i].x,testset[i].edge_index,testset[i].edge_attr)
    out = model.predict_link(z, edge_index).view(-1)
    out = out.sigmoid().cpu().detach().numpy()
    # print(out)
    score = roc_auc_score(y.cpu().numpy(),out)
    test_scores.append(score)
  return sum(test_scores)/len(test_scores)


In [ ]:
train_scores=[]
test_scores=[]
for epoch in range(1,101):
  loss = train(model,trainset,optim,criterion)
  train_scores.append(loss)
  if epoch%1==0:
    test_score=test(model,testset)
    test_scores.append(test_score)
    print(f'TEST SCORE : {test_score}')
  print(f'Epoch : {epoch} && Loss : {loss}')

TEST SCORE : 0.5442800768680849
Epoch : 1 && Loss : 71009566720000.0
TEST SCORE : 0.5531904901984002
Epoch : 2 && Loss : 358806097756160.0
TEST SCORE : 0.5622923873232001
Epoch : 3 && Loss : 1792183283744768.0
TEST SCORE : 0.5370337274713081
Epoch : 4 && Loss : 1.2711700889337856e+16
TEST SCORE : 0.5906880691280794
Epoch : 5 && Loss : 1.2352412916383744e+16
TEST SCORE : 0.5788750604913461
Epoch : 6 && Loss : 2678333015851008.0
TEST SCORE : 0.5709006796961377
Epoch : 7 && Loss : 1530734800011264.0
TEST SCORE : 0.554165030681634
Epoch : 8 && Loss : 467201073086464.0
TEST SCORE : 0.5612070959193792
Epoch : 9 && Loss : 726827584716800.0
TEST SCORE : 0.5644180652875768
Epoch : 10 && Loss : 377436659253248.0
TEST SCORE : 0.5606523606563377
Epoch : 11 && Loss : 231702395879424.0
TEST SCORE : 0.5656267234168095
Epoch : 12 && Loss : 98444928614400.0
TEST SCORE : 0.5492814674400196
Epoch : 13 && Loss : 31400885485568.0
TEST SCORE : 0.5243942341445741
Epoch : 14 && Loss : 10292578746368.0
TEST SC